In [1]:
"""
We use following lines because we are running on Google Colab
If you are running notebook on a local computer, you don't need this cell
"""
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/finch/tensorflow1/free_chat/chinese/main')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%tensorflow_version 1.x
!pip install texar

In [3]:
import tensorflow as tf
import texar.tf as tx

import numpy as np
import pprint
import logging

from pathlib import Path
from texar.tf.modules import TransformerEncoder, TransformerDecoder

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.test.is_gpu_available())

TensorFlow Version 1.15.0
GPU Enabled: True


In [0]:
# stream data from text files
def data_generator(f_paths, params):
  char2idx = params['char2idx']
  for f_path in f_paths:
    with open(f_path) as f:
      print('Reading', f_path)
      for line in f:
        line = line.rstrip()
        sp = line.split('|')
        if len(sp) == 2:
          source, target = sp
        else:
          continue
        source = [char2idx.get(c, len(char2idx)) for c in list(source)]
        target = [char2idx.get(c, len(char2idx)) for c in list(target)]
        if len(source) > params['max_len']:
          source = source[:params['max_len']]
        if len(target) > params['max_len']:
          target = target[:params['max_len']]
        target_in = [1] + target
        target_out = target + [2]
        if len(source) > 0:
          yield (source, (target_in, target_out))

In [0]:
def dataset(is_training, params):
  _shapes = ([None], ([None], [None]))
  _types = (tf.int32, (tf.int32, tf.int32))
  _pads = (0, (0, 0))
  
  if is_training:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['train_paths'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['buffer_size'])
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  else:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator(params['test_paths'], params),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.padded_batch(params['batch_size'], _shapes, _pads)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  
  return ds

In [0]:
def clip_grads(loss):
    variables = tf.trainable_variables()
    pprint.pprint(variables)
    grads = tf.gradients(loss, variables)
    clipped_grads, _ = tf.clip_by_global_norm(grads, params['clip_norm'])
    return zip(clipped_grads, variables)

In [0]:
def forward(words, labels, mode):
    words_len = tf.count_nonzero(words, 1, dtype=tf.int32)
    
    is_training = (mode == tf.estimator.ModeKeys.TRAIN)
    batch_sz = tf.shape(words)[0]
    
  
    with tf.variable_scope('Embedding'):
        embedding = tf.Variable(np.load('../vocab/char.npy'),
                                dtype=tf.float32,
                                name='fasttext_vectors')
        embedding = tf.concat([tf.zeros(shape=[1, params['embed_dim']]), embedding[1:, :]], axis=0)
        x = tf.nn.embedding_lookup(embedding, words)
        pos_embedder = tx.modules.SinusoidsPositionEmbedder(
            position_size = params['max_len'] + 1,
            hparams = config_model.position_embedder_hparams)
        x = (x * config_model.hidden_dim ** 0.5) + pos_embedder(sequence_length=words_len)


    with tf.variable_scope('Encoder'):
        encoder = TransformerEncoder(hparams=config_model.encoder)
        enc_out = encoder(inputs=x, sequence_length=words_len)
    
    
    with tf.variable_scope('Decoder'):
        decoder = TransformerDecoder(vocab_size=len(params['char2idx'])+1,
                                     output_layer=tf.transpose(embedding, (1, 0)),
                                     hparams=config_model.decoder)
        
        if is_training or (mode == tf.estimator.ModeKeys.EVAL):
            dec_inputs, dec_outputs = labels
            dec_seq_len = tf.count_nonzero(dec_inputs, 1, dtype=tf.int32)
            dec_inputs = tf.nn.embedding_lookup(embedding, dec_inputs)
            dec_inputs = (dec_inputs * config_model.hidden_dim ** 0.5) + pos_embedder(sequence_length=dec_seq_len)

            outputs = decoder(
                memory=enc_out,
                memory_sequence_length=words_len,
                inputs=dec_inputs,
                decoding_strategy='train_greedy',
                mode=tf.estimator.ModeKeys.TRAIN
            )
            
            return outputs.logits
        else:
            start_tokens = tf.fill([batch_sz], 1)

            def _embedding_fn(x, y):
                x_w_embed = tf.nn.embedding_lookup(embedding, x)
                y_p_embed = pos_embedder(y)
                return x_w_embed * config_model.hidden_dim ** 0.5 + y_p_embed

            predictions = decoder(
                memory=enc_out,
                memory_sequence_length=words_len,
                beam_width=params['beam_width'],
                length_penalty=params['length_penalty'],
                start_tokens=start_tokens,
                end_token=2,
                embedding=_embedding_fn,
                max_decoding_length=params['max_len'],
                mode=tf.estimator.ModeKeys.PREDICT)
            
            return predictions['sample_id'][:, :, :params['top_k']]

In [0]:
def model_fn(features, labels, mode, params):
    logits_or_ids = forward(features, labels, mode)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode, predictions=logits_or_ids)
    
    dec_inputs, dec_outputs = labels
    if (params['label_smoothing'] <= .0) or (mode == tf.estimator.ModeKeys.EVAL):
      loss_op = tf.contrib.seq2seq.sequence_loss(logits = logits_or_ids,
                                                 targets = dec_outputs,
                                                 weights = tf.to_float(tf.sign(dec_outputs)))
    else:
      loss_op = tf.losses.softmax_cross_entropy(onehot_labels = tf.one_hot(dec_outputs, len(params['char2idx'])+1),
                                                logits = logits_or_ids,
                                                weights = tf.to_float(tf.sign(dec_outputs)),
                                                label_smoothing = params['label_smoothing'],)
      
    if mode == tf.estimator.ModeKeys.TRAIN:
        global_step=tf.train.get_or_create_global_step()
        
        decay_lr = tf.train.exponential_decay(
            params['lr'], global_step, 1000, .99)
        
        train_op = tf.train.AdamOptimizer(decay_lr).apply_gradients(
            clip_grads(loss_op), global_step=global_step)
        
        hook = tf.train.LoggingTensorHook({'lr': decay_lr}, every_n_iter=100)
        
        return tf.estimator.EstimatorSpec(
            mode=mode, loss=loss_op, train_op=train_op, training_hooks=[hook],)
      
    if mode == tf.estimator.ModeKeys.EVAL:
      return tf.estimator.EstimatorSpec(mode=mode, loss=loss_op)

In [0]:
def get_vocab(f_path):
  word2idx = {}
  with open(f_path) as f:
    for i, line in enumerate(f):
      line = line.rstrip('\n')
      word2idx[line] = i
  return word2idx

In [0]:
def pad(test_strs):
  max_len = max([len(test_str) for test_str in test_strs])
  for test_str in test_strs:
    if len(test_str) < max_len:
      test_str += ['<pad>'] * (max_len - len(test_str))


def minimal_test(estimator):
  test_strs = [
    '你好',
    '早上好',
    '晚上好',
    '再见',
    '好久不见',
    '想死你了',
    '谢谢你',
    '爱你',
    '你好厉害啊',
    '你叫什么',
    '你几岁了',
    '现在几点',
    '今天天气怎么样',
    '我们现在在哪里',
    '讲个笑话',
    '你会几种语言呀',
    '你觉得我帅吗',
    '讨厌的周一',
    '好烦啊',
    '天气真好',
    '今天好冷',
    '今天好热',
    '下雨了',
    '风好大',
    '终于周五了',
    '我想去K歌',
  ]
  test_strs = [list(test_str) for test_str in test_strs]
  pad(test_strs)
  test_arrs = [[params['char2idx'].get(c, len(params['char2idx'])) for c in test_str] for test_str in test_strs]
  predicted = list(estimator.predict(tf.estimator.inputs.numpy_input_fn(
    x = np.asarray(test_arrs), shuffle = False)))
  predicted = np.asarray(predicted)
  print('-'*12)
  print('minimal test')
  for i, test_str in enumerate(test_strs):
    print('Q:', ' '.join([c for c in test_str if c != '<pad>']))
    for j in range(params['top_k']):
      sent = ' '.join([params['idx2char'].get(idx, '<unk>') for idx in predicted[i, :, j] if (idx != 0 and idx != 2)])
      print('A{}:'.format(j+1), sent)
    print()
  print('-'*12)

In [0]:
def is_increasing(history: list) -> bool:
  history = history[-(params['num_patience']+1):]
  for i in range(1, len(history)):
    if history[i-1] >= history[i]:
      return False
  return True  

In [0]:
class config_model:
    hidden_dim = 300
    dropout_rate = .2
    num_blocks = 3

    position_embedder_hparams = {
        'dim': hidden_dim
    }

    encoder = {
            'dim': hidden_dim,
            'embedding_dropout': dropout_rate,
            'residual_dropout': dropout_rate,
            'num_blocks': num_blocks,
            'multihead_attention': {
                'dropout_rate': dropout_rate,
                'num_heads': 8,
                'output_dim': hidden_dim,
                'use_bias': True,
            },
            'initializer': {
                'type': 'variance_scaling_initializer',
                'kwargs': {
                    'scale': 1.0,
                    'mode': 'fan_avg',
                    'distribution': 'uniform',
                },
            },
            'poswise_feedforward': tx.modules.default_transformer_poswise_net_hparams(
                output_dim=hidden_dim),
        }

    decoder = {
        'dim': hidden_dim,
        'embedding_dropout': dropout_rate,
        'residual_dropout': dropout_rate,
        'num_blocks': num_blocks,
        'multihead_attention': {
            'dropout_rate': dropout_rate,
            'num_heads': 8,
            'output_dim': hidden_dim,
            'use_bias': True,
        },
        'initializer': {
            'type': 'variance_scaling_initializer',
            'kwargs': {
                'scale': 1.0,
                'mode': 'fan_avg',
                'distribution': 'uniform',
            },
        },
        'poswise_feedforward': tx.modules.default_transformer_poswise_net_hparams(
            output_dim=hidden_dim),
        'output_layer_bias': True,
    }

params = {
    'model_dir': '../model/transformer',
    'log_path': '../log/transformer.txt',
    'train_paths': ['../data/core.txt'] + ['../data/reduced.txt'],
    'test_paths': ['../data/core.txt'],
    'vocab_path': '../vocab/char.txt',
    'max_len': 30,
    'embed_dim': config_model.hidden_dim,
    'beam_width': 5,
    'top_k': 3,
    'length_penalty': .6,
    'label_smoothing': .1,
    'lr': 7e-4,
    'clip_norm': 5.,
    'buffer_size': 180000,
    'batch_size': 64,
    'num_patience': 5,
}

In [0]:
params['char2idx'] = get_vocab(params['vocab_path'])
params['idx2char'] = {idx: char for char, idx in params['char2idx'].items()}

In [14]:
# Create directory if not exist
Path(os.path.dirname(params['log_path'])).mkdir(exist_ok=True)
Path(params['model_dir']).mkdir(exist_ok=True, parents=True)

# Logging
logger = logging.getLogger('tensorflow')
logger.setLevel(logging.INFO)
fh = logging.FileHandler(params['log_path'])
logger.addHandler(fh)

# Create an estimator
estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  model_dir=params['model_dir'],
  config=tf.estimator.RunConfig(
    save_checkpoints_steps=params['buffer_size']//params['batch_size'], keep_checkpoint_max=1),
  params=params)

best_ppl = 10000.
history_ppl = []
tf.enable_eager_execution()

while True:
  estimator.train(input_fn=lambda: dataset(is_training=True, params=params))

  minimal_test(estimator)
  
  loss = estimator.evaluate(input_fn=lambda: dataset(is_training=False, params=params))['loss']
  ppl = np.exp(loss)
  logger.info("Perplexity: {:.3f}".format(ppl))
  history_ppl.append(ppl)

  if ppl < best_ppl:
    best_ppl = ppl
  logger.info("Best Perplexity: {:.3f}".format(best_ppl))

  if len(history_ppl) > params['num_patience'] and is_increasing(history_ppl):
    logger.info("Perplexity not improved over {} epochs, Early Stop".format(params['num_patience']))
    break
  
  # BLEU
  labels = [label for _, (_, label) in dataset(is_training=False, params=params)]
  labels = [j for i in labels for j in i.numpy()]
  labels = [[params['idx2char'].get(idx, '<unk>') for idx in arr if (idx!=0 and idx!=2)] for arr in labels]

  preds = list(estimator.predict(input_fn=lambda: dataset(is_training=False, params=params)))
  assert len(labels) == len(preds)
  preds = [[params['idx2char'].get(idx, '<unk>') for idx in arr[:, 0] if (idx!=0 and idx!=2)] for arr in preds]

  logger.info("BLEU: {:.1f}".format(tx.evals.corpus_bleu_moses(list_of_references=[[l] for l in labels], hypotheses=preds)))

INFO:tensorflow:Using config: {'_model_dir': '../model/transformer', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 2812, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f2d70218fd0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized auto

KeyboardInterrupt: ignored